In [8]:
import yfinance as yf
import numpy as np
import pandas as pd

In [9]:
def backtest(data, signal, initial_capital=100000):
    capital = initial_capital
    position = 0
    entry_price = 0.0
    trades = []
    equity_curve = []
    current_sl = 0.0
    current_tp = 0.0

    for i in range(len(data)):
        price = float(data.iloc[i]["Close"])
        low_price = float(data.iloc[i]["Low"])
        high_price = float(data.iloc[i]["High"])
        sig = int(signal[i])

        if position > 0:
            sl_hit = low_price <= current_sl
            tp_hit = high_price >= current_tp

            if sl_hit:
                pnl = position * (current_sl - entry_price)
                capital += position * current_sl
                trades.append(pnl)
                position = 0
                equity_curve.append(capital)
                continue

            elif tp_hit:
                pnl = position * (current_tp - entry_price)
                capital += position * current_tp
                trades.append(pnl)
                position = 0
                equity_curve.append(capital)
                continue

        elif position < 0:
            sl_hit = high_price >= current_sl
            tp_hit = low_price <= current_tp

            if sl_hit:
                pnl = abs(position) * (entry_price - current_sl)
                capital += pnl
                trades.append(pnl)
                position = 0
                equity_curve.append(capital)
                continue

            elif tp_hit:
                pnl = abs(position) * (entry_price - current_tp)
                capital += pnl
                trades.append(pnl)
                position = 0
                equity_curve.append(capital)
                continue

        if position > 0 and sig == -1:
            pnl = position * (price - entry_price)
            capital += position * price
            trades.append(pnl)
            position = 0
        elif position < 0 and sig == 1:
            pnl = abs(position) * (entry_price - price)
            capital += pnl
            trades.append(pnl)
            position = 0


        if position == 0:
            if sig == 1:
                qty = int(capital // price)
                if qty > 0:
                    position = qty
                    entry_price = price
                    capital -= qty * price
                    current_sl = entry_price * 0.95
                    current_tp = entry_price * 1.10

            elif sig == -1:
                qty = int(4*capital // price)
                if qty > 0:
                    position = -qty
                    entry_price = price

                    current_sl = entry_price * 1.05
                    current_tp = entry_price * 0.90
        equity = capital
        if position > 0:
            equity += position * price
        elif position < 0:
            unrealized_pnl = abs(position) * (entry_price - price)
            equity += unrealized_pnl

        equity_curve.append(equity)

    final_price = float(data.iloc[-1]["Close"])
    if position != 0:
        if position > 0:
            pnl = position * (final_price - entry_price)
            capital += position * final_price
        elif position < 0:
            pnl = abs(position) * (entry_price - final_price)
            capital += pnl
        trades.append(pnl)

    trades = np.array(trades)
    wins = trades[trades > 0]
    win_rate = (len(wins) / len(trades) * 100) if len(trades) else 0
    total_pnl = capital - initial_capital
    return_pct = (total_pnl / initial_capital) * 100

    equity_curve = np.array(equity_curve)
    drawdown = equity_curve / np.maximum.accumulate(equity_curve) - 1
    max_drawdown = drawdown.min() * 100

    print(f"Net Profit: {round(total_pnl, 2)}")
    print(f"Returns: {round(return_pct, 2)}%")
    print(f"Sharpe Ratio: {sharpe_ratio(equity_curve)}")
    print(f"Total Trades: {len(trades)}")
    print(f"Total Wins: {len(wins)}")
    print(f"Total Losses: {len(trades) - len(wins)}")
    print(f"Max Drawdown (%): {round(max_drawdown, 2)}")

Rest all code is copied from question 1.
Though I am not confident about logical correctness of my code because in short trading i haven't added time limit to hold positions.

In [10]:
# Previous data was loaded to get indicator signals which need previous days' data.
def get_data():
  data = yf.download('TCS.NS', start='2022-12-01', end='2023-12-31', interval='1d', auto_adjust=False, progress=False)
  return data[["Open", "High", "Low", "Close", "Volume"]].dropna()
def get_real_data():
  data = yf.download('TCS.NS', start='2022-12-10', end='2023-12-31', interval='1d', auto_adjust=False, progress=False)
  return data[["Open", "High", "Low", "Close", "Volume"]].dropna()

In [11]:
def sharpe_ratio(equity_curve, risk_free_rate=0.0):
  equity_series = pd.Series(equity_curve)
  daily_returns = equity_series.pct_change().dropna()
  num_days = len(daily_returns)
  mean_daily_return = daily_returns.mean()
  std_daily_return = daily_returns.std()

  if std_daily_return == 0:
      return 0.0

  sharpe_ratio = np.sqrt(num_days) * (mean_daily_return - risk_free_rate) / std_daily_return
  return sharpe_ratio

In [12]:
def RSI(data):
  delta = data['Close'].diff()
  gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
  loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
  rs = gain / loss
  data['RSI'] = 100 - (100 / (1 + rs))
  return data

In [13]:
def MACD(data):
  ema_fast = data['Close'].ewm(span=12, adjust=False).mean()
  ema_slow = data['Close'].ewm(span=26, adjust=False).mean()
  data['MACD'] = ema_fast - ema_slow
  data['MACD_Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()
  data['MACD_Histogram'] = data['MACD'] - data['MACD_Signal']
  return data

In [14]:
data = get_data()
data = RSI(data)
data = MACD(data)

data['RSI_Vote'] = np.where(data['RSI'] < 30, 1, np.where(data['RSI'] > 70, -1, 0))
data['MACD_Vote'] = np.where(data['MACD_Histogram'] > 0, 1, np.where(data['MACD_Histogram'] < 0, -1, 0))
data['MACD_Cross_Up'] = (data['MACD_Histogram'] > 0) & (data['MACD_Histogram'].shift(1) <= 0)
data['MACD_Cross_Down'] = (data['MACD_Histogram'] < 0) & (data['MACD_Histogram'].shift(1) >= 0)
data['RSI_Recover'] = (data['RSI'] > 30) & (data['RSI'].shift(1) <= 30)
data['Signal'] = 0


data['EMA_Short'] = data['Close'].ewm(span=20, adjust=False).mean()
data['EMA_Long'] = data['Close'].ewm(span=50, adjust=False).mean()
data['Crossover_Up'] = (data['EMA_Short'] > data['EMA_Long']) & (data['EMA_Short'].shift(1) <= data['EMA_Long'].shift(1))
data['Crossover_Down'] = (data['EMA_Short'] < data['EMA_Long']) & (data['EMA_Short'].shift(1) >= data['EMA_Long'].shift(1))

#buy = data['MACD_Cross_Up'] | (data['RSI'] < 30) | data['Crossover_Up']
#sell = data['MACD_Cross_Down'] | (data['RSI'] > 70) | data['Crossover_Down']
buy = (data['RSI_Vote'] == 1) | (data['MACD_Vote'] == 1)
sell = (((data['RSI_Vote'] == -1) & (data['MACD_Vote'] == -1)) | (data['RSI'] > 80))

data.loc[buy, 'Signal'] = 1
data.loc[sell, 'Signal'] = -1
real_data = get_real_data()
signals = np.array(data['Signal'].loc[real_data.index].values)

print(f"Number of non-zero signals: {np.sum(np.abs(signals))}")
print(f"Number of buy signals (1): {np.sum(signals == 1)}")
print(f"Number of sell signals (-1): {np.sum(signals == -1)}")
backtest(real_data, signals)

Number of non-zero signals: 170
Number of buy signals (1): 161
Number of sell signals (-1): 9
Net Profit: 40809.91
Returns: 40.81%
Sharpe Ratio: 1.9558014848485803
Total Trades: 12
Total Wins: 8
Total Losses: 4
Max Drawdown (%): -10.09


/tmp/ipython-input-407450796.py:11: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(data.iloc[i]["Close"])
/tmp/ipython-input-407450796.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  low_price = float(data.iloc[i]["Low"])
/tmp/ipython-input-407450796.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  high_price = float(data.iloc[i]["High"])
/tmp/ipython-input-407450796.py:95: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  final_price = float(data.iloc[-1]["Close"])
